In [11]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [12]:
!kaggle datasets download -d crowdflower/first-gop-debate-twitter-sentiment

Dataset URL: https://www.kaggle.com/datasets/crowdflower/first-gop-debate-twitter-sentiment
License(s): CC-BY-NC-SA-4.0


In [13]:
!unzip first-gop-debate-twitter-sentiment.zip

Archive:  first-gop-debate-twitter-sentiment.zip
  inflating: Sentiment.csv           
  inflating: database.sqlite         


In [14]:
import numpy as np
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, SimpleRNN
from sklearn.model_selection import train_test_split
import re

In [15]:
df = pd.read_csv(r"/content/Sentiment.csv")

In [16]:
df.head()

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13871 entries, 0 to 13870
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         13871 non-null  int64  
 1   candidate                  13775 non-null  object 
 2   candidate_confidence       13871 non-null  float64
 3   relevant_yn                13871 non-null  object 
 4   relevant_yn_confidence     13871 non-null  float64
 5   sentiment                  13871 non-null  object 
 6   sentiment_confidence       13871 non-null  float64
 7   subject_matter             13545 non-null  object 
 8   subject_matter_confidence  13871 non-null  float64
 9   candidate_gold             28 non-null     object 
 10  name                       13871 non-null  object 
 11  relevant_yn_gold           32 non-null     object 
 12  retweet_count              13871 non-null  int64  
 13  sentiment_gold             15 non-null     obj

In [18]:
data = df[['text','sentiment']]

In [19]:
data.head()

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [20]:
data = data[data['sentiment'].apply(lambda x:x.lower().strip()) != 'neutral']

In [21]:
data.head()

,text,sentiment
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",Positive
6,RT @warriorwoman91: I liked her and was happy ...,Negative


In [22]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply(lambda x: re.sub(r'(#|@)\w*', '', x))
data['text'] = data['text'].apply(lambda x: re.sub("https?:\/\/\S+", '', x))
data['text'] = data['text'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '',x))
data['text'] = data['text'].apply(lambda x: re.sub(r'^rt ', '', x))

In [23]:
data.head()

,text,sentiment
1,didnt catch the full last night here are som...,Positive
3,that carly fiorina is trending hours after h...,Positive
4,w delivered the highest ratings in the hist...,Positive
5,on my first day i will rescind every illegal...,Positive
6,i liked her and was happy when i heard she wa...,Negative


In [24]:
data['text'] = data['text'].apply(lambda x: re.sub(r'^rt ', '',x))

In [25]:
data.head()

,text,sentiment
1,didnt catch the full last night here are som...,Positive
3,that carly fiorina is trending hours after h...,Positive
4,w delivered the highest ratings in the hist...,Positive
5,on my first day i will rescind every illegal...,Positive
6,i liked her and was happy when i heard she wa...,Negative


In [26]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [27]:
data['text']

,text
1,didnt catch the full last night here are som...
3,that carly fiorina is trending hours after h...
4,w delivered the highest ratings in the hist...
5,on my first day i will rescind every illegal...
6,i liked her and was happy when i heard she wa...
...,...
13866,love to see men who will never be faced with ...
13867,who thought huckabee exceeded their expectati...
13868,as president i will always tell the truth an...
13869,donald trump says that he doesnt have time f...


In [28]:
max_features = 2500
tokenizer = Tokenizer(num_words= max_features, split=' ',lower = True, oov_token='UNK')
tokenizer.fit_on_texts(data['text'].values)
x = tokenizer.texts_to_sequences(data['text'].values)
x = pad_sequences(x)
print(x[0])
print(x.shape[1])


[   0    0    0    0    0    0    0    0    0    0    0    0  115 2059
    2  642   36   52  222   33  193    5    1  163 1535   11 1153 1231]
28


In [29]:
emdedding_dimension = 128
lstm_units = 196
model = Sequential()
model.add(Embedding(max_features, emdedding_dimension, input_length=x.shape[1]))
model.add(SimpleRNN(lstm_units))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
y = pd.get_dummies(data['sentiment']).values

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [30]:
x_train, X_test, y_train , y_test = train_test_split(x,y, test_size = 0.2, random_state = 42)
print(x_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(8583, 28) (8583, 2)
(2146, 28) (2146, 2)


In [31]:
model.fit(x_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
269/269 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.8034 - loss: 0.4819
Epoch 2/10
269/269 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8576 - loss: 0.3334
Epoch 3/10
269/269 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9125 - loss: 0.2283
Epoch 4/10
269/269 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9366 - loss: 0.1627
Epoch 5/10
269/269 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9542 - loss: 0.1311
Epoch 6/10
269/269 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9593 - loss: 0.1205
Epoch 7/10
269/269 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9526 - loss: 0.1260
Epoch 8/10
269/269 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9642 - loss: 0.1051
Epoch 9/10
269/269 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9655 - loss: 0.0934
Epoch 10/10
269/269 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9681 - loss: 0.0893
